# Prepare Environment

**Install pyspark package**

The first step before running spark is to install the ``pyspark`` package, because the package is not included in the default Google Colab setup.
This step needs to be done every time we open a new session

In [ ]:
!pip install pyspark

**b. Create spark session**

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('DataFrame Basics').getOrCreate()



We will perform some simple data engineering with sales data. First we will load the clean data, and then we will use 'dirty' data and do some cleansing and enrichment.

#Latihan data

Data covid, ada 2 file

- data harian covid : 
seq, dateRep, cases, deaths, geoId, cnum_14days_per_100K

- data referensi : 
countriesAndTerritories, geoId, countryterritoryCode, popData2019, continentExp

</p>
<i>Note : The formula to calculate the 14-day cumulative number of reported COVID-19 cases per 100 000 population is  (New cases over 14 day period)/Population)*100 000. </i>


##1. Data Harian Covid

Data utama
Ada data duplikat, ada data incomplete, ada data yang tidak standar

In [ ]:
#Data harian covid 

!wget https://www.dropbox.com/s/9le71jjcpdqzzk9/datacovid_dup1.csv

In [ ]:
df = spark.read.csv("datacovid_dup1.csv",header=True,inferSchema=True)

In [ ]:
df.show()
df.printSchema()

+---+----------+-----+------+-----+--------------------+
|_c0|   dateRep|cases|deaths|geoId|cnum_14days_per_100K|
+---+----------+-----+------+-----+--------------------+
|  0|08/11/2020|126.0|   6.0|   AF|          3.65650829|
|  1|07/11/2020| 58.0|   2.0|   AF|          3.53821723|
|  2|06/11/2020| 40.0|   0.0|   AF|           3.5461033|
|  3|05/11/2020|121.0|   6.0|   AF|          3.74588377|
|  4|04/11/2020| 86.0|   4.0|   AF|          3.78268543|
|  5|03/11/2020| 95.0|   3.0|   AF|          3.78794281|
|  6|02/11/2020|132.0|   5.0|   AF|          3.76691329|
|  7|01/11/2020| 76.0|   0.0|   AF|          3.57501889|
|  8|31/10/2020|157.0|   4.0|   AF|          3.55398937|
|  9|30/10/2020|123.0|   3.0|   AF|          3.26483343|
| 10|29/10/2020|  0.0|   0.0|   AF|          2.94150452|
| 11|28/10/2020|113.0|   7.0|   AF|          3.02562261|
| 12|27/10/2020|199.0|   8.0|   AF|          2.90207416|
| 13|26/10/2020| 65.0|   3.0|   AF|          2.71806583|
| 14|25/10/2020| 81.0|   4.0|  

In [ ]:
df.createOrReplaceTempView("datacovid")


##Data Referensi 

- countryterritoryCode : nama negara/wilayah
- popData2019 : populasi tahun 2019
- continentExp : nama benua

In [ ]:
!wget https://www.dropbox.com/s/attxr4nalhpakti/countries3.csv

In [ ]:
dfr = spark.read.csv("countries3.csv",header=True,inferSchema=True)

In [ ]:
dfr.show(5)
dfr.printSchema()

+---+-----------------------+-----+--------------------+-----------+------------+
|_c0|countriesAndTerritories|geoId|countryterritoryCode|popData2019|continentExp|
+---+-----------------------+-----+--------------------+-----------+------------+
|  0|           Cote_dIvoire|   CI|                 CIV|2.5716554E7|      Africa|
|  1|                Lebanon|   LB|                 LBN|  6855709.0|        Asia|
|  2|                Armenia|   AM|                 ARM|  2957728.0|      Europe|
|  3|                Bolivia|   BO|                 BOL|1.1513102E7|     America|
|  4|          Guinea_Bissau|   GW|                 GNB|  1920917.0|      Africa|
+---+-----------------------+-----+--------------------+-----------+------------+
only showing top 5 rows

root
 |-- _c0: integer (nullable = true)
 |-- countriesAndTerritories: string (nullable = true)
 |-- geoId: string (nullable = true)
 |-- countryterritoryCode: string (nullable = true)
 |-- popData2019: double (nullable = true)
 |-- cont

##Pertanyaan yang harus dijawab : 
1. Ada berapa total hari pelaporan?
2. Dari tanggal berapa sampai tanggal berapa?
<br><br>
3. Ada berapa data invalid?
4. Ada berapa data duplikat?
<br><br>
5. Berapa total kasus positif di Timor Leste ?
6. Berapa total kematian di Inggris?
7. Benua mana yang rasio kematiannya (terhadap jumlah kasus) paling besar?